In [2]:
import elasticsearch
from datetime import datetime

In [3]:
country="chile"
from_="2021-04-01"
to_="2021-04-30"
media_outlets=["elllanquihue","elaustral","laestrelladechiloe","elheraldoaustral","radiosago",
               "elrepuertero","elvacanudo","elhuemul","seminariolocal","elquellonino","elinsular",
               "radiopudeto","radioacogida","elcalbucano","segundos33",
               "prensadelestuario","fresiaahora","soychiloe"]

keyword=""
simple_keyword=False

In [4]:
IP = "search.sophia2.org"
PORT = 9200
USER= "elastic"
PASS= "dummy_elastic"

In [5]:
#NO MODIFICAR

es = elasticsearch.Elasticsearch(
    IP,
    #port=PORT,
    http_auth=(USER, PASS)
)

In [6]:
#NO MODIFICAR
match=""
if (simple_keyword):
    match="match"
else:
    match="match_phrase"

query = { 
    "bool": { 
      #"must": [
      #  {match: { "text":keyword}}

      #],
    "filter": [
        {"range": {
      "date": {
        "gte": from_,
        "lt": to_
      }}},
        { "term":  { "country": country }},
        { "terms":  { "media_outlet": media_outlets }} 
    ]
    }  
}

In [7]:
res = es.search(index="news", query=query, size=10000)
print("Son %d noticias encontradas..." % res['hits']['total']['value'])

Son 6773 noticias encontradas...


In [8]:
import pandas as pd

data = {'id_news':[],'country':[],'media_outlet':[],'url':[],'title':[],'text':[],'date':[]}

df = pd.DataFrame(data)  
  
for hit in res['hits']['hits']:
    id_news = hit['_source']['id_news']
    country = hit['_source']['country']
    media_outlet = hit['_source']['media_outlet']
    url = hit['_source']['url']
    title = hit['_source']['title']
    text = hit['_source']['text']
    date = hit['_source']['date']
    
    new_row = {'id_news':id_news, 'country':country, 'media_outlet':media_outlet, 'url':url, 'title':title, 'text':text, 'date':date}
    
    df = df.append(new_row, ignore_index=True)

df

,id_news,country,media_outlet,url,title,text,date
0,21889860.0,chile,radiosago,https://www.radiosago.cl/se-intensifica-el-par...,Se intensifica el paro portuario en Puerto Mon...,Los trabajadores agrupados en la Unión Portua...,2021-04-27
1,21915643.0,chile,radiosago,https://www.radiosago.cl/confirman-octavo-caso...,Confirman octavo caso de Hantavirus en lo que ...,En la comuna de Panguipulli se registró un nue...,2021-04-28
2,21915810.0,chile,radiosago,https://www.radiosago.cl/la-region-de-los-lago...,La región de Los Lagos reportó 283 contagios d...,Así lo dio a conocer la Seremi de Salud de la ...,2021-04-25
3,21915811.0,chile,radiosago,https://www.radiosago.cl/puerto-octay-y-puerto...,Puerto Octay y Puerto Varas avanzarán a Fase 3...,"La Subsecretaria de Salud Pública, Paula Daza,...",2021-04-26
4,21915812.0,chile,radiosago,https://www.radiosago.cl/244-nuevos-contagios-...,244 nuevos contagios y 4 fallecidos por corona...,Según las cifras dadas a conocer por la autori...,2021-04-26
...,...,...,...,...,...,...,...
6768,47512676.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/0...,Monster Hunter Rise ahora trae la cacería de m...,Tras el éxito comercial y críticas de Monster ...,2021-04-05
6769,47512677.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/0...,Satélites artificiales: el impacto en vigilanc...,De los más de 2.000 satélites artificiales que...,2021-04-05
6770,47512678.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/0...,Parche de Cyberpunk 2077 arregla bugs,La actualización 1.2 de Cyberpunk 2077 está di...,2021-04-05
6771,47512679.0,chile,laestrelladechiloe,https://www.laestrellachiloe.cl/impresa/2021/0...,Among Us estrena su primer mapa nuevo,Ya ha sido liberado el primer mapa nuevo del j...,2021-04-05


In [9]:
df['url'].duplicated().any()

True

In [10]:
df = df.drop_duplicates(subset='url', keep='first')

In [43]:
nombre_archivo="01-04-21.csv"
df.to_csv(nombre_archivo)

In [11]:
df['title'][0]

'Se intensifica el paro portuario en Puerto Montt en segundo día de protesta'